In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import math as m
df = pd.read_csv('af_interval.csv')
n=117
amount_int=8


In [2]:
def cumfreq(df):
    df['cumabs']=np.cumsum(df['abs'])
    df['cumotn']=np.cumsum(df['otn'])
    return df
df=cumfreq(df)
df

dif 1       dif 2  abs       otn  cumabs    cumotn
0  -5932305.0   5964453.0   58  0.495726      58  0.495726
1   5964453.0  17861211.0   30  0.256410      88  0.752137
2  17861211.0  29757969.0    7  0.059829      95  0.811966
3  29757969.0  41654727.0   15  0.128205     110  0.940171
4  41654727.0  53551485.0    3  0.025641     113  0.965812
5  53551485.0  65448243.0    1  0.008547     114  0.974359
6  65448243.0  77345001.0    1  0.008547     115  0.982906
7  77345001.0  89241759.0    2  0.017094     117  1.000000

In [3]:
mid = (df['dif 2'][0]-df['dif 1'][0])/2
df['mid'] = [df['dif 1'][i]+mid for i in range (8)]
df

dif 1       dif 2  abs       otn  cumabs    cumotn         mid
0  -5932305.0   5964453.0   58  0.495726      58  0.495726     16074.0
1   5964453.0  17861211.0   30  0.256410      88  0.752137  11912832.0
2  17861211.0  29757969.0    7  0.059829      95  0.811966  23809590.0
3  29757969.0  41654727.0   15  0.128205     110  0.940171  35706348.0
4  41654727.0  53551485.0    3  0.025641     113  0.965812  47603106.0
5  53551485.0  65448243.0    1  0.008547     114  0.974359  59499864.0
6  65448243.0  77345001.0    1  0.008547     115  0.982906  71396622.0
7  77345001.0  89241759.0    2  0.017094     117  1.000000  83293380.0

In [24]:
df.to_csv('af_interval.csv', sep = ',',index=False)

## Вычисление условных вариант:

In [4]:
if amount_int % 2 != 0:
    C = df['mid'][amount_int/2]
else:
    C = df['mid'][amount_int/2] if df['abs'][amount_int/2]>df['abs'][amount_int/2-1] else df['mid'][amount_int/2-1]
C


35706348.0

In [5]:
h = df['dif 2'][0]-df['dif 1'][0]
df['cond'] = [(df['mid'][i]-C)/h for i in range (8)]
df

dif 1       dif 2  abs       otn  cumabs    cumotn         mid  cond
0  -5932305.0   5964453.0   58  0.495726      58  0.495726     16074.0  -3.0
1   5964453.0  17861211.0   30  0.256410      88  0.752137  11912832.0  -2.0
2  17861211.0  29757969.0    7  0.059829      95  0.811966  23809590.0  -1.0
3  29757969.0  41654727.0   15  0.128205     110  0.940171  35706348.0   0.0
4  41654727.0  53551485.0    3  0.025641     113  0.965812  47603106.0   1.0
5  53551485.0  65448243.0    1  0.008547     114  0.974359  59499864.0   2.0
6  65448243.0  77345001.0    1  0.008547     115  0.982906  71396622.0   3.0
7  77345001.0  89241759.0    2  0.017094     117  1.000000  83293380.0   4.0

## Вычисление моментов:

In [6]:
cond_em = [np.sum(pow(df['cond'],i)*df['otn']) for i in range(1,5)]
cond_em

[-1.9230769230769236,
 5.957264957264954,
 -14.076923076923084,
 49.54700854700852]

In [7]:
cent_em = [0]
cent_em.append((cond_em[1]-pow(cond_em[0],2))*pow(h,2))
cent_em.append((cond_em[2]-3*cond_em[0]*cond_em[1]+2*pow(cond_em[0],3))*pow(h,3))
cent_em.append((cond_em[3]-4*cond_em[0]*cond_em[2]+6*pow(cond_em[0],2)*cond_em[1]-3*pow(cond_em[0],4))+pow(h,4))
cent_em

[0, 319728386409399.6, 1.021714161141922e+22, 2.003154788687194e+28]

In [8]:
list = {'Order':['first','second','third','fourth'],'cond EM':cond_em,'center EM': cent_em}
em=pd.DataFrame(list)
em

Order    cond EM     center EM
0   first  -1.923077  0.000000e+00
1  second   5.957265  3.197284e+14
2   third -14.076923  1.021714e+22
3  fourth  49.547009  2.003155e+28

## По стандартным формулам:

In [9]:
sample_mean = np.sum(df['mid']*df['otn']) 
sample_mean

12827967.230769213

In [10]:
sample_var=  np.sum(pow(df['mid']-sample_mean,2)*df['otn'])
sample_var

319728386409399.5

## С помощью условных вариант:

In [11]:
cond_mean = np.sum(df['cond']*df['otn'])
sample_mean = cond_mean*h+C 
sample_mean

12827967.230769224

In [12]:
sample_var = pow(h,2)*(np.sum(pow(df['cond'],2)*df['otn'])-pow(cond_mean,2))
sample_var

319728386409399.6

## Исправленные параметры:

In [13]:
fix_var =sample_var* n/(n-1)
fix_var

322484665602584.1

In [14]:
fix_SD = np.sqrt(fix_var)
fix_SD

17957858.04606396

## Кэффициенты ассиметрии и эксцесса:

In [15]:
SD = np.sqrt(sample_var)
skew = cent_em[2]/pow(SD,3)
skew

1.7871360644070562

In [16]:
kurt = cent_em[3]/pow(SD,4) -3
kurt

-2.8040469093254865

## Мода и медиана:

In [17]:
check = sum(df['abs'])/2
maxabs = 0
for i in range (8):
    if df['abs'][i] > maxabs :
        index = i
        maxabs = df['abs'][i]
df['dif 1'][index], df['dif 2'][index]

(-5932305.0, 5964453.0)

In [18]:
p = df['otn'][index]
if index == 0:
    p1 = 0
    p2 = df['otn'][index+1]
elif index == amount_int-1:
    p1 = df['otn'][index-1]
    p2 = 0
mode = df['dif 1'][index] + h*(p-p1)/((p-p1)+(p-p2))
mode

2091089.9302325575

In [22]:
index=0
for i in range (8):
    if df['cumotn'][i] <= 0.5:
        index +=1
df['dif 1'][index], df['dif 2'][index]

(5964453.0, 17861211.0)

In [23]:
med = df['dif 1'][index] + (h/df['otn'][index])*(0.5-df['cumotn'][index-1])
med

6162732.300000002

## Kоэффициент вариации

In [21]:
CV = SD/sample_mean * 100
CV

139.39036525444433